In [1]:
import pickle
import en_core_web_sm
import spacy
import random
from spacy import displacy

In [2]:
filename = '../training_data'
infile = open(filename,'rb')
TRAIN_DATA = pickle.load(infile)
infile.close()

In [3]:
nlp=spacy.load('en_core_web_sm')

In [4]:
def train_model(train_data):
    if 'ner' not in nlp.pipe_names:
        ner=nlp.create_pipe('ner')
        nlp.add_pipe(ner,last=True)
    else:
        ner = nlp.get_pipe("ner")
    for _,annotation in train_data:
        for ent in annotation['entities']:
            ner.add_label(ent[2])
    


    other_pipes = [pipe for pipe in nlp.pipe_names if pipe !='ner']
    # only train NER
    with nlp.disable_pipes(*other_pipes):
        optimizer=nlp.entity.create_optimizer()
        for itn in range(10):
            random.shuffle(train_data)
            losses = {}
            index=0
            for text,annotations in train_data:
                try:
                    nlp.update(
                        [text],  # batch of texts
                        [annotations],  # batch of annotations
                        drop=0.2,
                        sgd=optimizer,  # dropout - make it harder to memorise data
                        losses=losses)
                except Exception as e:
                    pass
            print(losses)
        
    

In [5]:
train_model(TRAIN_DATA)

{'ner': 1381.324449294471}
{'ner': 1170.4888928397304}
{'ner': 1231.9665315469674}
{'ner': 1226.776977592835}
{'ner': 1161.7646310767825}
{'ner': 1129.5093030281682}
{'ner': 1071.9676770484311}
{'ner': 1033.9307587074159}
{'ner': 1013.7330287257421}
{'ner': 1119.0388946704081}


In [6]:
nlp.to_disk('nlp_model')

In [7]:
nlp = spacy.load('nlp_model')
doc = nlp("Shareholders of Torrent Power will consider a proposal for a dividend of Rs 2.20 per share for 2016-17 at its annual general meeting on August 1")
for ent in doc.ents:
    print(ent.text,ent.label_)

In [15]:
displacy.serve(doc,style='ent')


Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.
